In [1]:
import math
import scipy as sp
import matplotlib.pyplot as plt
from scipy.stats import bernoulli, uniform, chi2
import numpy as np
from scipy.stats.mstats import gmean
from numpy.testing import assert_allclose
from utils import sprt_mart, eb_selector, psi_E, v_i
np.random.seed(123456789)

In [2]:
stratum_1 = np.random.normal(loc = 0.5, scale = 0.05, size = 200)
stratum_2 = np.random.normal(loc = 0.6, scale = 0.05, size = 100)
strata = [stratum_1, stratum_2]
lam = np.array([0.5])

In [8]:
N = np.array([len(x) for x in strata])
K = len(strata)
w = N/np.sum(N)
gamma = 1
marts = [np.ones(x) for x in N]
a = [(gamma/(np.arange(N[k]) + 1)) * np.cumsum(lam*strata[k] - psi_E(lam)*v_i(strata[k])) + (1-gamma)*w[k] for k in np.arange(K)]
running_n = np.zeros(K)
running_a = np.ones(K)
#record which strata are pulled from
selected_strata = np.zeros(np.sum(N))
mart = np.ones(np.sum(N))
i = 0
while any(running_n < (N-1)):
    next_stratum = eb_selector(running_a = running_a, running_n = running_n, lam = lam, N = N, gamma = gamma)
    selected_strata[i] = next_stratum
    running_n[next_stratum] += 1
    running_a[next_stratum] = a[next_stratum][int(running_n[next_stratum])]
    i += 1
#TODO: need to define the martingale given selected_strata, then optimize over the null using greedy algorithm